In [ ]:
import pandas as pd
from general_preprocess import *

In [ ]:
inter_info = pd.read_csv("../ml-1m/ratings.dat", sep="::", engine="python", encoding="latin-1", names=["user_id:token", "item_id:token", "rating:float", "timestamp:float"])

inter_info.head()

In [ ]:
item_info = pd.read_csv("../ml-1m/movies.dat", sep="::", engine="python", encoding="latin-1", index_col=0, names=["item_id:token", "movie_title:token_seq", "class:token_seq"])
item_info[["movie_title:token_seq", "release_year:token"]] = item_info["movie_title:token_seq"].str.rsplit(" ", n=1, expand=True)
item_info["release_year:token"] = item_info["release_year:token"].str[1:-1]

item_preprocess(item_info, inter_info)

item_info.head()

In [ ]:
user_info = pd.read_csv("../ml-1m/users.dat", sep="::", engine="python", index_col=0, names=["user_id:token", "gender:token", "age:token", "occupation:token", "zip_code:token"])

user_preprocess(user_info, item_info, inter_info)

user_info.head()

In [ ]:
user_info.to_csv("../ml-1m/ml-1m.user", sep="\t")
item_info.to_csv("../ml-1m/ml-1m.item", sep="\t")
inter_info.to_csv("../ml-1m/ml-1m.inter", index=None, sep="\t")